In [1]:
import os
import subprocess
import math
from pathlib import Path
import shutil
import numpy as np
import random
import copy
from scipy.stats import pearsonr
from pprint import pprint
from argparse import ArgumentParser

In [5]:
# --- Configuration ---
RDIR = "/home/luis_ubuntu/unixdir/Peptide_Binding/smm_nested_cv/"
DDIR = "/home/luis_ubuntu/unixdir/Peptide_Binding/Data"

#alleles = [
#    "A0101", "A0202", "A2402", "A3001", "A3101", "A6901",
#    "B0801", "B3501", "B4403", "B5801", "A0201", "A2301",
#    "A2403", "A3002", "A6801", "B0702", "B2705", "B4402", "B5701"
#]

alleles = [
    "A2403", "A3002", "A6801", "B0702"
]

lambdas = [0.02, 0.08, 0.1]
epis = [0.01, 0.04, 0.08]
folds = range(5)

In [6]:
# --- Pearson correlation calculator ---
def pearson_from_pairs(pairs):
    n = len(pairs)
    if n == 0:
        return 0.0, float("inf")
    
    x = [p[0] for p in pairs]
    y = [p[1] for p in pairs]
    
    x0 = sum(x) / n
    y0 = sum(y) / n
    
    t = nx = ny = err = 0.0
    for i in range(n):
        dx = x[i] - x0
        dy = y[i] - y0
        t += dx * dy
        nx += dx * dx
        ny += dy * dy
        err += (x[i] - y[i]) ** 2
    
    if nx * ny == 0:
        pcc = 0.0
    else:
        pcc = t / math.sqrt(nx * ny)
    
    mse = err / n
    return pcc, mse

def make_and_enter(dir_path):
    path = Path(dir_path)
    path.mkdir(exist_ok=True)
    os.chdir(path)

def concat_train_files(n_list, allele, DDIR, held_out_fold):
    output_file = Path(f"conc_f00{held_out_fold}")
    if output_file.exists():
        return output_file  # Return existing file without rewriting
    
    with open(output_file, "w") as outfile:
        for n in n_list:
            file_path = Path(f"{DDIR}/{allele}/c00{n}")
            with open(file_path) as infile:
                outfile.writelines(infile.readlines())

def run_training_and_evaluation(RDIR, train_file, eval_file, mat_file, pred_file, _lambda, _epsilon):
    # Run training
    if not Path(mat_file).exists():
        with open(mat_file, "w") as fout:
            subprocess.run(
                [
                    "python", f"{RDIR}/smm_gradient_descent.py",
                    "-l", str(_lambda),
                    "-epi", str(_epsilon),
                    "-t", train_file
                ],
                stdout=fout,
                stderr=subprocess.DEVNULL,
                env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
            )

    # Run evaluation
    if not Path(pred_file).exists():
        with open(pred_file, "w") as fout:
            subprocess.run(
                [
                    "python", f"{RDIR}/pep2score.py",
                    "-mat", mat_file,
                    "-f", eval_file
                ],
                stdout=fout,
                stderr=subprocess.DEVNULL,
                env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
            )

def collect_outer_preds(allele):
    preds = []
    preds_for_file = []
    for path in Path('.').rglob('*.outer_pred'):
        with open(path) as pf:
            for line in pf:
                if "#" not in line and line.strip():
                    try:
                        parts = line.strip().split()
                        preds_for_file.append(f"{parts[0]} {parts[1]} {parts[2]}\n")
                    except:
                        continue

    # Save all collected predictions to a single file
    output_file = Path(f"{allele}_prediction")
    with open(output_file, "w") as out:
        out.writelines(preds_for_file)
    
    print(f"Saved {len(preds_for_file)} predictions to {output_file}")

In [7]:
os.chdir(RDIR)
# --- Loop over all alleles ---
for allele in alleles:
    make_and_enter(f"{allele}.res")

    # --- Outer Validation loop ---
    for n in folds:
        inner_loop_files = [0, 1, 2, 3, 4]
        inner_loop_files.remove(n)
    
        make_and_enter(f"validation_set_{n}")

        # copy the validation files and create the new concatenated training files
        for m in inner_loop_files:
            # Copy the evaluation file
            eval_file = f"{DDIR}/{allele}/c00{m}"
            shutil.copy(eval_file, f"c00{m}")
            # Create the concatenated trining files
            train_files = inner_loop_files.copy()
            train_files.remove(m)

            concat_train_files(train_files, allele, DDIR, m)
        
        best_pcc = -1000
        best_model = ""
        best_lambda = ""
        best_epsilon = ""
        
    # --- Hyperparameters loop ---    
        for l in lambdas:
            make_and_enter(f"l.{l}")
                
            for epi in epis:
                make_and_enter(f"epi.{epi}")
    
                preds = []

                # --- Inner CV loop ---
                for m in inner_loop_files:
                    
                    # Define the files to start running
                    mat_file = f"mat.{m}"
                    pred_file = f"c00{m}.pred"
                    
                    #eval_file = f"../../c00{m}"
                    eval_file = f"{RDIR}{allele}.res/validation_set_{n}/c00{m}"
                    #train_file = f"../../conc_f00{m}"
                    train_file = f"{RDIR}{allele}.res/validation_set_{n}/conc_f00{m}"
                    
                    # Run training and evalutation
                    
                    run_training_and_evaluation(RDIR, train_file, eval_file, mat_file=mat_file, pred_file=pred_file, _lambda=l, _epsilon=epi)
                    
                    with open(pred_file) as pf:
                        for line in pf:
                            if "#" not in line and line.strip():
                                try:
                                    parts = line.strip().split()
                                    preds.append((float(parts[1]), float(parts[2])))
                                except:
                                    continue
                
                # Compute PCC and MSE
                pcc, mse = pearson_from_pairs(preds)
                eval_output = f"{allele}; Outer validation set = {n}, lambda = {l} epsilon = {epi} PCC {pcc:.5f} MSE {mse:.5f}"
                print(eval_output)

                if pcc > best_pcc:
                    best_pcc = pcc
                    best_lambda = l
                    best_epsilon = epi
                    best_model = f"lambda {best_lambda} epsilon {best_epsilon}"
                    
                
                # Leave epi.{epi}
                os.chdir("..")
        
            # Leave l.{l}
            os.chdir("..")

        # Final result
        print("\nBest model for allele", allele,"and Outer validation set '", n, "', :", best_model, "with correlation", f"{best_pcc:.5f}\n")

        # Copy the final evaluation file
        final_eval_file = f"{DDIR}/{allele}/c00{n}"
        shutil.copy(final_eval_file, f"final_evaluation_c00{n}")
        
        # Copy the full training file to use when the best HP are chosen
        full_train_file = f"{DDIR}/{allele}/f00{n}"
        shutil.copy(full_train_file, f"final_training_f00{n}")

        # Define the files to start running
        mat_file = f"mat.{n}"
        pred_file = f"c00{n}.outer_pred"
                    
        eval_file = f"final_evaluation_c00{n}"
        train_file = f"final_training_f00{n}"

        # Run training and evaluation
        run_training_and_evaluation(RDIR, train_file, eval_file, mat_file=mat_file, pred_file=pred_file, _lambda=best_lambda, _epsilon=best_epsilon)
        
        # Leave validation_set_{n}
        os.chdir("..")

    # Concatenate predictions for allele and get the final measure
    collect_outer_preds(allele)

    pairs = []
    with open(f"{allele}_prediction") as f:
        for line in f:
            parts = line.strip().split()
            try:
                x = float(parts[1])  # second column
                y = float(parts[2])  # third column
                pairs.append((x, y))
            except (IndexError, ValueError):
                continue
                
    pcc, mse = pearson_from_pairs(pairs)
    eval_output = f"Final prediciton for allele:{allele}; PCC {pcc:.5f} MSE {mse:.5f}"
    print(eval_output)
    
    # Leave {allele}.res
    os.chdir("..")
                    

A2403; Outer validation set = 0, lambda = 0.02 epsilon = 0.01 PCC 0.36564 MSE 0.07315
A2403; Outer validation set = 0, lambda = 0.02 epsilon = 0.04 PCC 0.30818 MSE 0.11507
A2403; Outer validation set = 0, lambda = 0.02 epsilon = 0.08 PCC 0.28429 MSE 0.14964
A2403; Outer validation set = 0, lambda = 0.08 epsilon = 0.01 PCC 0.36578 MSE 0.07299
A2403; Outer validation set = 0, lambda = 0.08 epsilon = 0.04 PCC 0.30892 MSE 0.11411
A2403; Outer validation set = 0, lambda = 0.08 epsilon = 0.08 PCC 0.28565 MSE 0.14732
A2403; Outer validation set = 0, lambda = 0.1 epsilon = 0.01 PCC 0.36580 MSE 0.07294
A2403; Outer validation set = 0, lambda = 0.1 epsilon = 0.04 PCC 0.30914 MSE 0.11381
A2403; Outer validation set = 0, lambda = 0.1 epsilon = 0.08 PCC 0.28607 MSE 0.14657

Best model for allele A2403 and Outer validation set ' 0 ', : lambda 0.1 epsilon 0.01 with correlation 0.36580

A2403; Outer validation set = 1, lambda = 0.02 epsilon = 0.01 PCC 0.40947 MSE 0.06618
A2403; Outer validation set = 